In [2]:
# CNN Implementation for CIFAR Datasets with 3 convulational blocks

!pip install -q tensorflow matplotlib seaborn scikit-learn opencv-python

# Mounting Google Drive for saving models and results
from google.colab import drive
drive.mount('/content/drive')

# Create a directory to save models and results
import os
save_dir = '/content/drive/MyDrive/CIFAR_CNN_Project (3 blocks)'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Checking GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Set memory growth to avoid OOM errors
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("Memory growth set to True")

# Import all required libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2
import time
import pandas as pd

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Function to load and preprocess CIFAR datasets
def load_and_preprocess_data(dataset='cifar10', validation_split=0.1):

    # Load the dataset
    if dataset == 'cifar10':
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        num_classes = 10
    else:  # cifar100
        (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
        num_classes = 100

    # Create a validation set
    val_size = int(len(x_train) * validation_split)
    indices = np.random.permutation(len(x_train))
    train_indices, val_indices = indices[val_size:], indices[:val_size]

    x_val, y_val = x_train[val_indices], y_train[val_indices]
    x_train, y_train = x_train[train_indices], y_train[train_indices]

    # Convert data to float32 and normalize
    x_train = x_train.astype('float32') / 255.0
    x_val = x_val.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Print data ranges to verify normalization
    print(f"Training data range: {x_train.min()} to {x_train.max()}")
    print(f"Validation data range: {x_val.min()} to {x_val.max()}")
    print(f"Test data range: {x_test.min()} to {x_test.max()}")

    # Convert labels to one-hot encoding
    y_train = to_categorical(y_train, num_classes)
    y_val = to_categorical(y_val, num_classes)
    y_test = to_categorical(y_test, num_classes)

    print(f'Dataset: {dataset}')
    print(f'Training set shape: {x_train.shape}, {y_train.shape}')
    print(f'Validation set shape: {x_val.shape}, {y_val.shape}')
    print(f'Test set shape: {x_test.shape}, {y_test.shape}')

    return x_train, y_train, x_val, y_val, x_test, y_test, num_classes

# Create a data augmentation generator
def create_data_generator():

    return ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        fill_mode='nearest'
    )

# Build a simpler and more stable CNN model
def build_model(input_shape, num_classes):

    # Use functional API instead of Sequential for better visualization support
    inputs = layers.Input(shape=input_shape)

    # First convolutional block
    x = layers.Conv2D(32, (3, 3), padding='same', name='conv1_1')(inputs)
    x = layers.BatchNormalization(name='bn1_1')(x)
    x = layers.Activation('relu', name='relu1_1')(x)
    x = layers.Conv2D(32, (3, 3), padding='same', name='conv1_2')(x)
    x = layers.BatchNormalization(name='bn1_2')(x)
    x = layers.Activation('relu', name='relu1_2')(x)
    x = layers.MaxPooling2D((2, 2), name='pool1')(x)
    x = layers.Dropout(0.2, name='dropout1')(x)

    # Second convolutional block
    x = layers.Conv2D(64, (3, 3), padding='same', name='conv2_1')(x)
    x = layers.BatchNormalization(name='bn2_1')(x)
    x = layers.Activation('relu', name='relu2_1')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', name='conv2_2')(x)
    x = layers.BatchNormalization(name='bn2_2')(x)
    x = layers.Activation('relu', name='relu2_2')(x)
    x = layers.MaxPooling2D((2, 2), name='pool2')(x)
    x = layers.Dropout(0.3, name='dropout2')(x)

    # Third convolutional block
    x = layers.Conv2D(128, (3, 3), padding='same', name='conv3_1')(x)
    x = layers.BatchNormalization(name='bn3_1')(x)
    x = layers.Activation('relu', name='relu3_1')(x)
    x = layers.Conv2D(128, (3, 3), padding='same', name='conv3_2')(x)
    x = layers.BatchNormalization(name='bn3_2')(x)
    x = layers.Activation('relu', name='relu3_2')(x)
    x = layers.MaxPooling2D((2, 2), name='pool3')(x)
    x = layers.Dropout(0.4, name='dropout3')(x)

    # Classification layers
    x = layers.GlobalAveragePooling2D(name='global_pool')(x)
    x = layers.Dense(512, name='dense1')(x)
    x = layers.BatchNormalization(name='bn_dense')(x)
    x = layers.Activation('relu', name='relu_dense')(x)
    x = layers.Dropout(0.5, name='dropout_dense')(x)
    outputs = layers.Dense(num_classes, activation='softmax', name='output')(x)

    model = models.Model(inputs=inputs, outputs=outputs, name='cifar_cnn')
    return model

# visualization functions
def visualize_feature_maps(model, image, layer_name):

    try:
        layer = None
        for l in model.layers:
            if l.name == layer_name:
                layer = l
                break

        if layer is None:
            print(f"Layer {layer_name} not found. Available layers:")
            for i, l in enumerate(model.layers):
                print(f"{i}: {l.name}")
            return None

        feature_model = models.Model(
            inputs=model.input,
            outputs=layer.output
        )

        # Get the feature maps for the input image
        image_batch = np.expand_dims(image, axis=0)
        feature_maps = feature_model.predict(image_batch)

        # Plot the feature maps
        fig, axes = plt.subplots(4, 8, figsize=(15, 8))
        axes = axes.flatten()

        # Display up to 32 feature maps (or fewer if there are less)
        num_maps = min(32, feature_maps.shape[-1])

        for i in range(num_maps):
            axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
            axes[i].set_title(f'Filter {i}')
            axes[i].axis('off')

        # Hide any unused subplots
        for i in range(num_maps, len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.suptitle(f'Feature Maps from Layer: {layer_name}')
        plt.subplots_adjust(top=0.9)
        return plt.gcf()

    except Exception as e:
        print(f"Error in visualize_feature_maps: {str(e)}")
        return None

# Fixed grad_cam function
def grad_cam(model, image, class_idx, layer_name=None):

    try:
        # If layer_name is not provided, find the last convolutional layer
        if layer_name is None:
            for layer in reversed(model.layers):
                if isinstance(layer, layers.Conv2D):
                    layer_name = layer.name
                    print(f"Automatically selected layer: {layer_name} for Grad-CAM")
                    break
            else:  # No convolutional layers found
                print("No convolutional layers found in the model. Cannot create Grad-CAM.")
                return None

        # Find the layer
        target_layer = None
        for layer in model.layers:
            if layer.name == layer_name:
                target_layer = layer
                break

        if target_layer is None:
            print(f"Layer {layer_name} not found. Available layers:")
            for i, layer in enumerate(model.layers):
                print(f"{i}: {layer.name}")
            return None

        # Create a model that maps the input image to the activations and output
        grad_model = models.Model(
            inputs=model.input,
            outputs=[target_layer.output, model.output]
        )

        # Compute the gradient of the class output with respect to the feature maps
        with tf.GradientTape() as tape:
            # Prepare the input image
            input_image = np.expand_dims(image, axis=0)
            input_image = tf.cast(input_image, tf.float32)
            tape.watch(input_image)

            # Get the model's output and feature maps
            conv_outputs, predictions = grad_model(input_image)
            class_output = predictions[:, class_idx]

        # Extract gradients
        grads = tape.gradient(class_output, conv_outputs)

        # Global average pooling of the gradients
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        # Weight the channels by the pooled gradients and sum
        conv_outputs = conv_outputs[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

        # Process the heatmap for visualization
        heatmap = np.maximum(heatmap, 0) / (np.max(heatmap) or 1e-10)  # Normalize
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

        # Apply colormap
        heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        # Convert RGB image to BGR for OpenCV
        image_bgr = (image * 255).astype(np.uint8)
        if len(image_bgr.shape) == 3 and image_bgr.shape[2] == 3:
            image_bgr = cv2.cvtColor(image_bgr, cv2.COLOR_RGB2BGR)

        # Overlay heatmap on original image
        superimposed_img = cv2.addWeighted(image_bgr, 0.6, heatmap_colored, 0.4, 0)

        # Convert back to RGB for matplotlib
        superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

        return superimposed_img

    except Exception as e:
        print(f"Error in grad_cam: {str(e)}")
        return None

# Function to plot training history
def plot_training_history(history):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)

    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    return fig

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names):

    y_true_classes = np.argmax(y_true, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    if len(class_names) > 20:
        from collections import Counter
        most_common_classes = [cls for cls, _ in Counter(y_true_classes).most_common(10)]

        mask = np.isin(y_true_classes, most_common_classes)
        y_true_classes_filtered = y_true_classes[mask]
        y_pred_classes_filtered = y_pred_classes[mask]

        selected_class_names = [class_names[i] for i in most_common_classes]

        cm = confusion_matrix(y_true_classes_filtered, y_pred_classes_filtered,
                             labels=most_common_classes)
    else:
        cm = confusion_matrix(y_true_classes, y_pred_classes)
        selected_class_names = class_names

    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Plot
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=selected_class_names, yticklabels=selected_class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Normalized Confusion Matrix')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    return plt.gcf()

# Function to analyze misclassified images
def analyze_misclassified(x_test, y_true, y_pred, class_names, num_images=10):

    y_true_classes = np.argmax(y_true, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    misclassified_indices = np.where(y_true_classes != y_pred_classes)[0]

    if len(misclassified_indices) == 0:
        print("No misclassified images found.")
        return None

    selected_indices = np.random.choice(
        misclassified_indices,
        size=min(num_images, len(misclassified_indices)),
        replace=False
    )

    num_cols = 5
    num_rows = (len(selected_indices) + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 3 * num_rows))
    axes = axes.flatten()

    for i, idx in enumerate(selected_indices):
        true_class = y_true_classes[idx]
        pred_class = y_pred_classes[idx]

        axes[i].imshow(x_test[idx])
        axes[i].set_title(f'True: {class_names[true_class]}\nPred: {class_names[pred_class]}')
        axes[i].axis('off')

    for i in range(len(selected_indices), len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle('Misclassified Images Analysis')
    plt.subplots_adjust(top=0.9)
    return fig

def get_cifar100_class_names():

    return [
        'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
        'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
        'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
        'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
        'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
        'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
        'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
        'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
        'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
        'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
        'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
        'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
        'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
        'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
        'worm'
    ]

# train_model function
def train_model(dataset='cifar10', batch_size=128, epochs=35, fine_tune=False, model_path=None):

    # Start timing
    start_time = time.time()

    # Load and preprocess data
    x_train, y_train, x_val, y_val, x_test, y_test, num_classes = load_and_preprocess_data(dataset)

    # Get class names
    if dataset == 'cifar10':
        class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                      'dog', 'frog', 'horse', 'ship', 'truck']
    else:  # cifar100
        class_names = get_cifar100_class_names()

    # Create data generator for augmentation
    datagen = create_data_generator()
    datagen.fit(x_train)

    # Colab-specific: Create TensorBoard callback
    tensorboard_callback = callbacks.TensorBoard(
        log_dir=f'{save_dir}/logs/{dataset}_{time.strftime("%Y%m%d-%H%M%S")}',
        histogram_freq=1
    )

    # Build or load the model
    if fine_tune and model_path and os.path.exists(model_path):
        print(f"Loading model from {model_path} for fine-tuning")
        try:
            # Load the base model
            base_model = models.load_model(model_path, compile=False)

            if hasattr(base_model, 'input_shape'):
                input_shape = base_model.input_shape[1:]
            else:
                input_shape = x_train.shape[1:]

            model = build_model(input_shape, num_classes)

            for i, layer in enumerate(model.layers[:-1]):
                if i < len(base_model.layers) and layer.name in [l.name for l in base_model.layers]:
                    base_layer = None
                    for bl in base_model.layers:
                        if bl.name == layer.name:
                            base_layer = bl
                            break

                    if base_layer and len(layer.get_weights()) > 0:
                        # Check if shapes match
                        base_weights = base_layer.get_weights()
                        if all(w1.shape == w2.shape for w1, w2 in zip(layer.get_weights(), base_weights)):
                            layer.set_weights(base_weights)
                            print(f"Transferred weights for layer: {layer.name}")

            print("Successfully created fine-tuned model with transferred weights")

        except Exception as e:
            print(f"Error loading or modifying pre-trained model: {str(e)}")
            print("Building a new model instead...")
            model = build_model(x_train.shape[1:], num_classes)
    else:
        print("Building a new model...")
        model = build_model(x_train.shape[1:], num_classes)

    # Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Model summary
    model.summary()

    # Print layer names for debugging
    print("\nLayer names in the model:")
    for i, layer in enumerate(model.layers):
        print(f"{i}: {layer.name}")

    # Callbacks
    callbacks_list = [
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        ),
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            filepath=f'{save_dir}/best_model_{dataset}.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        tensorboard_callback
    ]

    # Train the model with data generator
    train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
    steps_per_epoch = len(x_train) // batch_size

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=(x_val, y_val),
        callbacks=callbacks_list,
        verbose=1
    )

    # Print training time
    training_time = time.time() - start_time
    print(f"Training completed in {training_time/60:.2f} minutes")

    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
    print(f"Test accuracy: {test_acc:.4f}")

    # Make predictions
    y_pred = model.predict(x_test, verbose=1)

    # Classification report
    y_true_classes = np.argmax(y_test, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    print("\nClassification Report:")
    if dataset == 'cifar100':
        # For CIFAR-100, show only the most common classes in the report
        from collections import Counter
        most_common_classes = [cls for cls, _ in Counter(y_true_classes).most_common(20)]
        selected_class_names = [class_names[i] for i in most_common_classes]

        # Filter to most common classes for report
        mask = np.isin(y_true_classes, most_common_classes)
        if np.any(mask):  # Make sure we have at least one sample
            y_true_filtered = y_true_classes[mask]
            y_pred_filtered = y_pred_classes[mask]

            print(classification_report(
                y_true_filtered,
                y_pred_filtered,
                labels=most_common_classes,  # Important: specify the labels
                target_names=selected_class_names
            ))
        else:
            print("No samples found for the selected classes.")
    else:
        print(classification_report(y_true_classes, y_pred_classes,
                                  target_names=class_names))

    # Plot training history
    history_fig = plot_training_history(history)
    history_fig.savefig(f'{save_dir}/training_history_{dataset}.png')
    plt.close(history_fig)

    # Plot confusion matrix
    cm_fig = plot_confusion_matrix(y_test, y_pred, class_names)
    cm_fig.savefig(f'{save_dir}/confusion_matrix_{dataset}.png')
    plt.close(cm_fig)

    # Analyze misclassified images
    misclassified_fig = analyze_misclassified(x_test, y_test, y_pred, class_names)
    if misclassified_fig:
        misclassified_fig.savefig(f'{save_dir}/misclassified_{dataset}.png')
        plt.close(misclassified_fig)

    # Try/except blocks for visualizations to handle any errors gracefully
    try:
        # Find correctly classified images
        correct_indices = np.where(y_true_classes == y_pred_classes)[0]
        if len(correct_indices) > 0:
            # Select a few random correctly classified images
            sample_indices = np.random.choice(correct_indices, size=min(5, len(correct_indices)), replace=False)

            # For each selected image, generate visualizations
            for i, sample_idx in enumerate(sample_indices):
                sample_image = x_test[sample_idx]
                true_class = y_true_classes[sample_idx]

                print(f"\nGenerating visualizations for image {i+1}/{len(sample_indices)}, "
                      f"class: {class_names[true_class]}")

                try:
                    # 1. Feature map visualization
                    # Find a good conv layer to visualize (middle conv layer is usually good)
                    conv_layers = [layer.name for layer in model.layers if 'conv' in layer.name.lower()]
                    if conv_layers:
                        # Choose a middle conv layer
                        feature_layer = conv_layers[len(conv_layers) // 2]
                        print(f"Generating feature maps for layer: {feature_layer}")
                        feature_fig = visualize_feature_maps(model, sample_image, feature_layer)
                        if feature_fig:
                            feature_fig.savefig(f'{save_dir}/feature_maps_{dataset}_img{i+1}.png')
                            plt.close(feature_fig)
                except Exception as e:
                    print(f"Error generating feature maps: {str(e)}")

                try:
                    # 2. Grad-CAM visualization
                    print(f"Generating Grad-CAM with auto-selected layer")
                    gradcam_img = grad_cam(model, sample_image, true_class)

                    if gradcam_img is not None:
                        plt.figure(figsize=(10, 5))
                        plt.subplot(1, 2, 1)
                        plt.imshow(sample_image)
                        plt.title(f'Original Image: {class_names[true_class]}')
                        plt.axis('off')

                        plt.subplot(1, 2, 2)
                        plt.imshow(gradcam_img)
                        plt.title('Grad-CAM Visualization')
                        plt.axis('off')

                        plt.tight_layout()
                        plt.savefig(f'{save_dir}/gradcam_{dataset}_img{i+1}.png')
                        plt.close()
                except Exception as e:
                    print(f"Error generating Grad-CAM: {str(e)}")
    except Exception as e:
        print(f"Error during visualization: {str(e)}")

    # Save model summary to text file
    print("Saving model summary to text file...")
    with open(f'{save_dir}/model_summary_{dataset}.txt', 'w') as f:
        # Create a string representation of the model summary
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"Model summary saved to {save_dir}/model_summary_{dataset}.txt")

    # Save full training history to CSV for further analysis
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(f'{save_dir}/training_history_{dataset}.csv', index=False)

    # Calculate and report performance metrics on test set
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    # Overall metrics
    accuracy = accuracy_score(y_true_classes, y_pred_classes)

    # For multiclass metrics, use macro averaging to get an overall score
    precision = precision_score(y_true_classes, y_pred_classes, average='macro')
    recall = recall_score(y_true_classes, y_pred_classes, average='macro')
    f1 = f1_score(y_true_classes, y_pred_classes, average='macro')

    print("\nTest Set Performance Summary:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1 Score (macro): {f1:.4f}")

    # Save performance metrics to file
    with open(f'{save_dir}/performance_metrics_{dataset}.txt', 'w') as f:
        f.write(f"Dataset: {dataset}\n")
        f.write(f"Model: CNN with Batch Normalization\n")
        f.write(f"Training time: {training_time/60:.2f} minutes\n\n")
        f.write(f"Test accuracy: {accuracy:.4f}\n")
        f.write(f"Precision (macro): {precision:.4f}\n")
        f.write(f"Recall (macro): {recall:.4f}\n")
        f.write(f"F1 Score (macro): {f1:.4f}\n\n")

        f.write("Classification Report:\n")
        if dataset == 'cifar100':
            if np.any(mask):
                report = classification_report(
                    y_true_filtered,
                    y_pred_filtered,
                    labels=most_common_classes,
                    target_names=selected_class_names,
                    output_dict=False
                )
                f.write(report)
        else:
            report = classification_report(
                y_true_classes,
                y_pred_classes,
                target_names=class_names,
                output_dict=False
            )
            f.write(report)

    return model, history

# Main execution
if __name__ == "__main__":
    best_model_path = f'{save_dir}/best_model_cifar10.keras'

    # First train on CIFAR-10
    print("=== Training on CIFAR-10 ===")
    cifar10_model, _ = train_model(dataset='cifar10', epochs=30)

    # Fine-tune on CIFAR-100 using the best model from CIFAR-10 training
    print("\n=== Fine-tuning on CIFAR-100 ===")
    if os.path.exists(best_model_path):
        print(f"Using best model from {best_model_path} for fine-tuning")
        cifar100_model, _ = train_model(
            dataset='cifar100',
            epochs=35,
            fine_tune=True,
            model_path=best_model_path
        )
    else:
        print(f"Best model file {best_model_path} not found. Training CIFAR-100 from scratch.")
        cifar100_model, _ = train_model(dataset='cifar100', epochs=35)

    print("Training and evaluation completed!")

Mounted at /content/drive
TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth set to True
=== Training on CIFAR-10 ===
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step
Training data range: 0.0 to 1.0
Validation data range: 0.0 to 1.0
Test data range: 0.0 to 1.0
Dataset: cifar10
Training set shape: (45000, 32, 32, 3), (45000, 10)
Validation set shape: (5000, 32, 32, 3), (5000, 10)
Test set shape: (10000, 32, 32, 3), (10000, 10)
Building a new model...


Model: "cifar_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_1 (Conv2D)                     │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn1_1 (BatchNormalization)           │ (None, 32, 32, 32)          │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ relu1_1 (Activation)                 │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_2 (Conv2D)                     │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn1_2 (BatchNormalization)           │ (None, 32, 32, 32)          │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ relu1_2 (Activation)                 │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout1 (Dropout)                   │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_1 (Conv2D)                     │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn2_1 (BatchNormalization)           │ (None, 16, 16, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ relu2_1 (Activation)                 │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_2 (Conv2D)                     │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn2_2 (BatchNormalization)           │ (None, 16, 16, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ relu2_2 (Activation)                 │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout2 (Dropout)                   │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3_1 (Conv2D)                     │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn3_1 (BatchNormalization)           │ (None, 8, 8, 128)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ relu3_1 (Activation)                 │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3_2 (Conv2D)                     │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 362,026 (1.38 MB)

 Trainable params: 360,106 (1.37 MB)

...

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.1057 - loss: 3.9478
Epoch 1: val_accuracy improved from -inf to 0.24200, saving model to /content/drive/MyDrive/CIFAR_CNN_Project (3 blocks)/best_model_cifar100.keras
351/351 ━━━━━━━━━━━━━━━━━━━━ 49s 101ms/step - accuracy: 0.1059 - loss: 3.9465 - val_accuracy: 0.2420 - val_loss: 3.0154 - learning_rate: 0.0010
Epoch 2/35
  1/351 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2734 - loss: 3.0157

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.24200 to 0.24300, saving model to /content/drive/MyDrive/CIFAR_CNN_Project (3 blocks)/best_model_cifar100.keras
351/351 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2734 - loss: 3.0157 - val_accuracy: 0.2430 - val_loss: 3.0266 - learning_rate: 0.0010
Epoch 3/35
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2509 - loss: 2.9196
Epoch 3: val_accuracy did not improve from 0.24300
351/351 ━━━━━━━━━━━━━━━━━━━━ 23s 66ms/step - accuracy: 0.2509 - loss: 2.9194 - val_accuracy: 0.2392 - val_loss: 3.2893 - learning_rate: 0.0010
Epoch 4/35
  1/351 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3125 - loss: 2.5963
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: val_accuracy did not improve from 0.24300
351/351 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3125 - loss: 2.5963 - val_accuracy: 0.2380 - val_loss: 3.2727 - learning_rate: 0.0010
Epoch 5/35
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3111 

Model summary saved to /content/drive/MyDrive/CIFAR_CNN_Project (3 blocks)/model_summary_cifar100.txt

Test Set Performance Summary:
Accuracy: 0.4508
Precision (macro): 0.4786
Recall (macro): 0.4508
F1 Score (macro): 0.4403
Training and evaluation completed!
